# Айтем2айтем модель на основе implicit и матрицы нормированных каунтов продуктов 

- взвешенных экпоненциально по времени и обычных

01.02.2020

In [1]:
import os
import csv
import json
import random
import pickle
import glob
from datetime import datetime
from collections import defaultdict

import numpy as np
import pandas as pd
import implicit
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix
from tqdm import tqdm

from utils import (
    filter_user_item_pairs,
    get_user_item_matrix,
    normalized_average_precision
)

import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

%matplotlib inline

## Формируем юзер-айтем матрицу используя взвешенные по врмени каунты
- user_items_02_01_1 exp weight
- user_items_02_01_2 exp weight + fix 1./cnt_trans
- user_items_02_01_3 обычные каунты 1/cnt_trans (удаляем клиентов из check)

In [2]:
check_clients = [
    '001e840150', '00078c508d', '0008b2cb41', '00035a21d9', '0015aa77ce',
    '00184e8b0a', '000f3b9860', '001c2b565f', '000b0559be', '0016b0d9ad',
    '000a400848', '000940f00a', '000d599743', '00211fcfaa', '002283ef29',
    '001b8d6788', '00071890c8', '0007b4ca21', '000ddb6229', '0006b9ad75',
    '00042a927a', '00213be6fb', '0012d1d4aa', '00209f873d', '00184aab1b',
    '000702109b', '00031cbbe6', '001175d51b', '000bc820f6', '001c8984f0',
    '0010f1f8ca', '0009e6bafa', '000c049a1a', '0007667c60', '000ca87889',
    '00020e7b18', '001fb70769', '001566f916', '00010925a5', '001de90d21',
    '000efde438', '001392b297', '00134e091b', '0001f552b0', '00167a61e2',
    '000df9078a', '0018d2efac', '00047b3720', '00083b5b14', '0021e07838',
    '000c216adb', '001a2412c6', '0004315e57', '0019a16b6b', '000990be82',
    '0019fb86cb', '00184f3b10', '0010082ab3', '001dac232d', '0019ca361b',
    '0017fdd057', '000b9905d8', '0006fca4bf', '00140e5d34', '001d642f66',
    '001cef2991', '000bf8ff33', '00127b29bb', '0019e0f07d', '001c25b9e3',
    '000a9d12ff', '001f46aa2c', '000b45b7ac', '0018650c30', '0008244fb3',
    '0002ce2217', '00174b3561', '0004254599', '00068fd5dc', '001162084a',
    '000220a0a7', '000a00419c', '000bd5f2f1', '000012768d', '00022fd34f',
    '0018dea0ba', '0004e1e14e', '001d004e5e', '0004231e2a', '000bc94494',
    '00184df0c9', '000036f903', '0006f24465', '000ac12729', '0013c0cbab',
    '00177cee3e', '0020f90a83', '00065f11c7', '000f46bbfc', '00038f9200',
    '0017a7ebcb']
print(len(check_clients))

101


In [3]:
%%time
valid_time = datetime.strptime('2019-03-01 00:00:00', '%Y-%m-%d %H:%M:%S')
valid_chunks = [
    '../_processed_data/client_tr_history_4.tsv', 
    '../_processed_data/client_tr_history_0.tsv',
    '../_processed_data/client_tr_history_15.tsv', 
    '../_processed_data/client_tr_history_16.tsv', 
    '../_processed_data/client_tr_history_6.tsv'
]

train_chunks = sorted(glob.glob('../_processed_data/client_tr_history_*'))
train_chunks = [chunk for chunk in train_chunks if chunk not in valid_chunks]

product_cnt = defaultdict(int)
client_cnt = defaultdict(int)
client_product = defaultdict(float)

for chunk_file in train_chunks:
    with open(chunk_file, 'r') as chunk:
        for row in tqdm(chunk):
            client_id, transaction_history = row.split('\t')
            client_id, transaction_history = json.loads(client_id), json.loads(transaction_history)
            
            # drop clients from check file
            if client_id in check_clients:
                continue
                
            # tr history: dict -> list and sort it
            tr_history = [transaction_history[tr] for tr in transaction_history \
                          if datetime.strptime(
                                transaction_history[tr]['datetime'], '%Y-%m-%d %H:%M:%S'
                             ) < valid_time]
            sorted_transactions = sorted(tr_history, 
                                         key=lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S'))
            cnt_trans = len(sorted_transactions)
            
            if cnt_trans > 1:
                last_tr_dt = datetime.strptime(sorted_transactions[-1]['datetime'], '%Y-%m-%d %H:%M:%S')
                first_tr_dt = datetime.strptime(sorted_transactions[0]['datetime'], '%Y-%m-%d %H:%M:%S')
                coef1 = (valid_time - first_tr_dt).days + 1
                coef2 = (valid_time - last_tr_dt).days

                for transaction in sorted_transactions:
                    tr_dt = datetime.strptime(transaction['datetime'], '%Y-%m-%d %H:%M:%S')
                    cur_days_diff = (valid_time - tr_dt).days

                    for pr in transaction['products']:
                        product_cnt[pr['product_id']] += 1
                        client_cnt[client_id] += 1
                        client_product[(client_id, pr['product_id'])] += 1./cnt_trans
                
print(len(client_product), len(product_cnt), len(client_cnt))

20000it [00:10, 1985.97it/s]
20000it [00:10, 1974.05it/s]
20000it [00:10, 1936.72it/s]
20000it [00:10, 1971.28it/s]
20000it [00:10, 1913.18it/s]
20000it [00:09, 2070.32it/s]
20000it [00:10, 1948.74it/s]
20000it [00:10, 1974.09it/s]
20000it [00:11, 1804.11it/s]
20000it [00:09, 2008.36it/s]
162it [00:00, 1945.86it/s]
20000it [00:10, 1969.98it/s]
20000it [00:09, 2046.40it/s]
20000it [00:10, 1944.00it/s]
20000it [00:10, 1975.66it/s]
20000it [00:10, 1933.39it/s]

18859795 40510 281079
CPU times: user 2min 34s, sys: 2.34 s, total: 2min 37s
Wall time: 2min 33s


In [4]:
%%time
item_cnt_threshold = 2 # 02_01_1 02_01_2 02_01_3
client_cnt_threshold = 2 # 02_01_1 02_01_2 02_01_3

items_to_drop = [pr for pr in product_cnt if product_cnt[pr] < item_cnt_threshold]
users_to_drop = [cl for cl in client_cnt if client_cnt[cl] < client_cnt_threshold]

client_product_filtered = filter_user_item_pairs(client_product, users_to_drop, items_to_drop)
print(len(client_product), len(users_to_drop), len(items_to_drop))
print(len(client_product_filtered))

18859795 0 2302
18857493
CPU times: user 8.5 s, sys: 289 ms, total: 8.79 s
Wall time: 8.78 s


In [5]:
%%time
user_index, item_index, user_items = get_user_item_matrix(client_product_filtered, data_format='cnt')
print(len(user_index), len(item_index), user_items.shape)

281079 38208 (281079, 38208)
CPU times: user 14.8 s, sys: 158 ms, total: 15 s
Wall time: 15 s


In [6]:
with open('../_model_files/user_index_02_01_3.pckl', 'wb') as f:
    pickle.dump(user_index, f)
with open('../_model_files/item_index_02_01_3.pckl', 'wb') as f:
    pickle.dump(item_index, f)
with open('../_model_files/user_items_02_01_3.pckl', 'wb') as f:
    pickle.dump(user_items, f)

## Загружаем юзер-айтем матрицу, обучаем модель, проверка на check_queries

In [2]:
with open('../_model_files/user_index_02_01_3.pckl', 'rb') as f:
    user_index = pickle.load(f)
with open('../_model_files/item_index_02_01_3.pckl', 'rb') as f:
    item_index = pickle.load(f)
with open('../_model_files/user_items_02_01_3.pckl', 'rb') as f:
    user_items = pickle.load(f)

print(len(user_index), len(item_index), user_items.shape)

index_item = {v:k for k,v in item_index.items()}
print(len(index_item))

281079 38208 (281079, 38208)
38208


In [3]:
%%time
#model = implicit.nearest_neighbours.CosineRecommender(K=1)
model = implicit.nearest_neighbours.TFIDFRecommender(K=1)
model.fit(user_items.T)


CPU times: user 11.1 s, sys: 150 ms, total: 11.3 s
Wall time: 4.7 s


In [4]:
%%time
ap_values = []
queryset_file = '../retailhero-recommender-v2/data/check_queries.tsv'
num_items = len(item_index)
baseline_items = [
    '4009f09b04', '15ccaa8685', 'bf07df54e1', '3e038662c0', '4dcf79043e',
    'f4599ca21a', '5cb93c9bc5', '4a29330c8d', '439498bce2', '343e841aaa',
    '0a46068efc', 'dc2001d036', '31dcf71bbd', '5645789fdf', '113e3ace79',
    'f098ee2a85', '53fc95e177', '080ace8748', '4c07cb5835', 'ea27d5dc75',
    'cbe1cd3bb3', '1c257c1a1b', 'f5e18af323', '5186e12ff4', '6d0f84a0ac',
    'f95785964a', 'ad865591c6', 'ac81544ebc', 'de25bccdaf', 'f43c12d228',
]

with open(queryset_file) as fin:
    for line in tqdm(fin):        
        query_data, next_transaction = line.strip().split('\t')
        query_data, next_transaction = json.loads(query_data), json.loads(next_transaction)
        
        transaction_history = query_data['transaction_history']
        cnt_trans = len(transaction_history)
        #query_dt = datetime.strptime(query_data['query_time'], '%Y-%m-%dT%H:%M:%S')
            
        if cnt_trans > 1:
            sorted_transactions = sorted(transaction_history, 
                                         key=lambda x: datetime.strptime(x['datetime'], '%Y-%m-%dT%H:%M:%S'))                
            user_products = defaultdict(float)
            user_vector = np.zeros(shape=(1, num_items), dtype=np.float32)
                
            #last_tr_dt = datetime.strptime(sorted_transactions[-1]['datetime'], '%Y-%m-%dT%H:%M:%S')
            #first_tr_dt = datetime.strptime(sorted_transactions[0]['datetime'], '%Y-%m-%dT%H:%M:%S')
            #coef1 = (query_dt - first_tr_dt).days + 1
            #coef2 = (query_dt - last_tr_dt).days

            for transaction in sorted_transactions:
                #tr_dt = datetime.strptime(transaction['datetime'], '%Y-%m-%dT%H:%M:%S')
                #cur_days_diff = (query_dt - tr_dt).days

                for pr in transaction['products']:
                    user_products[pr['product_id']] += 1./cnt_trans#np.exp(-(cur_days_diff - coef2)/coef1)/cnt_trans

            for pr in user_products:
                if pr in item_index:
                    user_vector[0, item_index[pr]] = user_products[pr]

            raw_recs = model.recommend(userid=0, user_items=csr_matrix(user_vector), N=30, 
                                       filter_already_liked_items=False, recalculate_user=True)
            candidates = [index_item[ind] for (ind, score) in raw_recs]
        
        else:
            candidates = baseline_items.copy()
        
        ap = normalized_average_precision(next_transaction['product_ids'], candidates)
        ap_values.append(ap)
        
print(sum(ap_values) / len(ap_values))

101it [00:00, 1301.17it/s]

0.11239912021260304
CPU times: user 82.1 ms, sys: 199 µs, total: 82.3 ms
Wall time: 80.5 ms


## Разовая валидация на всех валидационных датасетах

In [5]:
def run_queries_implicit(model, check_file, item_index, index_item):
    """
    Take implicit model and make recommendations
    """
    ap_values = []
    queryset_file = check_file
    num_items = len(item_index)

    with open(queryset_file) as fin:
        for line in tqdm(fin):
            user_products = defaultdict(float)
            user_vector = np.zeros(shape=(1, num_items))

            query_data, next_transaction = line.strip().split('\t')
            query_data, next_transaction = json.loads(query_data), json.loads(next_transaction)
            
            cnt_trans = len(query_data['transaction_history'])
            for transaction in query_data['transaction_history']:
                store_id = transaction['store_id']
                for pr in transaction['products']:
                    if store_id in store_product_cnt:
                        if pr['product_id'] in store_product_cnt[store_id]:
                            coef = store_product_cnt[store_id][pr['product_id']]
                        else:
                            coef = 0.
                    else:
                        coef = 0.
                    user_products[pr['product_id']] += 1.*coef/cnt_trans

            for pr in user_products:
                if pr in item_index:
                    user_vector[0, item_index[pr]] = user_products[pr]

            raw_recs = model.recommend(userid=0, user_items=csr_matrix(user_vector), N=30, 
                                       filter_already_liked_items=False, recalculate_user=True)
            candidates_product = [index_item[ind] for (ind, score) in raw_recs]

            ap = normalized_average_precision(next_transaction['product_ids'], candidates_product)
            ap_values.append(ap)  
            
    return ap_values

In [6]:
%%time
valid_chunks = [
    '../_processed_data/client_tr_history_4.tsv', 
    '../_processed_data/client_tr_history_0.tsv',
    '../_processed_data/client_tr_history_15.tsv', 
    '../_processed_data/client_tr_history_16.tsv', 
    '../_processed_data/client_tr_history_6.tsv'
]
scores = []

for chunk_file in valid_chunks:
    ind = chunk_file.split('_')[-1].split('.')[0]
    valid_file_name = '../retailhero-recommender-v2/data/check_queries_01_11_{num}.tsv'.format(num=ind)
    
    ap_values = run_queries_implicit(model, valid_file_name, 
                                     item_index, index_item)
    score = sum(ap_values) / len(ap_values)
    scores.append(score)
    print(valid_file_name)
    print(round(score, 6))
    
print('\n*********************')
print('Avg score:', round(np.mean(scores), 6))

18877it [00:10, 1751.38it/s]
179it [00:00, 1778.96it/s]

../retailhero-recommender-v2/data/check_queries_01_11_4.tsv
0.11725


18874it [00:10, 1818.77it/s]
344it [00:00, 1712.06it/s]

../retailhero-recommender-v2/data/check_queries_01_11_0.tsv
0.11596


18918it [00:10, 1751.64it/s]
345it [00:00, 1707.10it/s]

../retailhero-recommender-v2/data/check_queries_01_11_15.tsv
0.117526


18883it [00:10, 1781.25it/s]
168it [00:00, 1670.58it/s]

../retailhero-recommender-v2/data/check_queries_01_11_16.tsv
0.117919


18904it [00:10, 1771.15it/s]

../retailhero-recommender-v2/data/check_queries_01_11_6.tsv
0.116431

*********************
Avg score: 0.117018
CPU times: user 53.2 s, sys: 173 ms, total: 53.4 s
Wall time: 53.2 s


## Сохраняем обученную модель для k=1

- model_implicit_nn_tfidf_02_01_1 на user_items_02_01_1
- model_implicit_nn_tfidf_02_01_2 на user_items_02_01_2
- model_implicit_nn_tfidf_02_01_3 на user_items_02_01_3

In [2]:
# read train data
with open('../_model_files/user_index_02_01_3.pckl', 'rb') as f:
    user_index = pickle.load(f)
with open('../_model_files/item_index_02_01_3.pckl', 'rb') as f:
    item_index = pickle.load(f)
with open('../_model_files/user_items_02_01_3.pckl', 'rb') as f:
    user_items = pickle.load(f)
print(len(user_index), len(item_index), user_items.shape)

index_item = {v:k for k,v in item_index.items()}

model = implicit.nearest_neighbours.TFIDFRecommender(K=1)
model.fit(user_items.T)
print('model trained...')

with open('../_model_files/model_implicit_nn_tfidf_02_01_3.pckl', 'wb') as f:
    pickle.dump(model, f)
print('model saved!')

281079 38208 (281079, 38208)



model trained...
model saved!


## Объединяем tfidf(k=1) на взвешенных каунтах и эвристику

- если использвоать user_items_02_01_2, то взвешенные каунты
- user_items_02_01_3 - обычные

In [2]:
%%time
with open('../_model_files/user_index_02_01_3.pckl', 'rb') as f:
    user_index = pickle.load(f)
with open('../_model_files/item_index_02_01_3.pckl', 'rb') as f:
    item_index = pickle.load(f)
with open('../_model_files/user_items_02_01_3.pckl', 'rb') as f:
    user_items = pickle.load(f)

print(len(user_index), len(item_index), user_items.shape)

index_item = {v:k for k,v in item_index.items()}
print(len(index_item))

with open('../_model_files/model_implicit_nn_tfidf_02_01_3.pckl', 'rb') as f:
    model_nn = pickle.load(f)

281079 38208 (281079, 38208)
38208
CPU times: user 89.5 ms, sys: 129 ms, total: 219 ms
Wall time: 217 ms


In [3]:
def blend_product_lists(*cands, num_candidates=30):
    cnt_dict = defaultdict(int)
    for cand in cands:
        for ind, pr in enumerate(cand):
            cnt_dict[pr] += ind + 1

    # add rank for items not any of set
    for cand in cands:
        cand = set(cand)

    for pr in cnt_dict:
        for cand in cands:
            if pr not in cand:
                cnt_dict[pr] += num_candidates

    sorted_list = sorted(cnt_dict.items(), key=lambda x: x[1], reverse=False)

    return [el[0] for el in sorted_list[:num_candidates]]

def get_user_vector(user_products_, item_index_, user_vector_):
    user_vector = user_vector_.copy()
    
    for pr in user_products_:
        if pr in item_index_:
            user_vector[0, item_index_[pr]] = user_products_[pr]
            
    return user_vector

In [8]:
%%time
ap_values = []
baseline_items = [
    '4009f09b04', '15ccaa8685', 'bf07df54e1', '3e038662c0', '4dcf79043e',
    'f4599ca21a', '5cb93c9bc5', '4a29330c8d', '439498bce2', '343e841aaa',
    '0a46068efc', 'dc2001d036', '31dcf71bbd', '5645789fdf', '113e3ace79',
    'f098ee2a85', '53fc95e177', '080ace8748', '4c07cb5835', 'ea27d5dc75',
    'cbe1cd3bb3', '1c257c1a1b', 'f5e18af323', '5186e12ff4', '6d0f84a0ac',
    'f95785964a', 'ad865591c6', 'ac81544ebc', 'de25bccdaf', 'f43c12d228',
]
queryset_file = '../retailhero-recommender-v2/data/check_queries.tsv'
num_items = len(item_index)

with open(queryset_file) as fin:
    for line in tqdm(fin):        
        query_data, next_transaction = line.strip().split('\t')
        query_data, next_transaction = json.loads(query_data), json.loads(next_transaction)
        
        transaction_history = query_data['transaction_history']
        cnt_trans = len(transaction_history)
        query_dt = datetime.strptime(query_data['query_time'], '%Y-%m-%dT%H:%M:%S')
        
        if cnt_trans > 1:
            sorted_transactions = sorted(transaction_history, 
                                         key=lambda x: datetime.strptime(x['datetime'], '%Y-%m-%dT%H:%M:%S'))                
            # several variants
            user_products = defaultdict(float)
            user_products_exp = defaultdict(float)
            user_vector = np.zeros(shape=(1, num_items), dtype=np.float32)
            
            # dt features
            last_tr_dt = datetime.strptime(sorted_transactions[-1]['datetime'], '%Y-%m-%dT%H:%M:%S')
            first_tr_dt = datetime.strptime(sorted_transactions[0]['datetime'], '%Y-%m-%dT%H:%M:%S')
            coef1 = (query_dt - first_tr_dt).days + 1
            coef2 = (query_dt - last_tr_dt).days

            for transaction in sorted_transactions:
                tr_dt = datetime.strptime(transaction['datetime'], '%Y-%m-%dT%H:%M:%S')
                cur_days_diff = (query_dt - tr_dt).days

                for pr in transaction['products']:
                    user_products[pr['product_id']] += 1./cnt_trans
                    user_products_exp[pr['product_id']] += np.exp(-(cur_days_diff - coef2)/coef1)/cnt_trans

            # get model nn candidates
            user_vector = get_user_vector(user_products, item_index, user_vector)
            raw_recs = model_nn.recommend(userid=0, user_items=csr_matrix(user_vector), N=30, 
                                       filter_already_liked_items=False, recalculate_user=True)
            candidates_nn = [index_item[ind] for (ind, score) in raw_recs]
            
            # get dummy candidates
            ups = sorted(user_products.items(), key=lambda x: x[1], reverse=True)
            candidates_dummy = [item[0] for item in ups if item[1] > 1./cnt_trans][:30]
            
            #ups = sorted(user_products.items(), key=lambda x: x[1], reverse=True)
            #candidates_dummy = [item[0] for item in ups if item[1] > 2.][:30]
            
            candidates_ = [candidates_nn, candidates_dummy]#, candidates_dummy2]
            candidates_ = blend_product_lists(*candidates_)
            
            candidates_set = set(candidates_)
            rec_len = len(candidates_)

            if rec_len < 30:
                items_add = [item for item in baseline_items if item not in candidates_set]
                candidates = candidates_ + items_add[:30 - rec_len]
            else:
                candidates = candidates_.copy()
        else:
            candidates = baseline_items.copy()   
        
        ap = normalized_average_precision(next_transaction['product_ids'], candidates)
        ap_values.append(ap)
        
print(sum(ap_values) / len(ap_values))

101it [00:00, 892.14it/s]

0.11412069271520268
CPU times: user 117 ms, sys: 168 µs, total: 117 ms
Wall time: 115 ms
